In [28]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('players.db')

In [29]:
def format_duration(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{int(hours):02d}h {int(minutes):02d}m {int(seconds):02d}s"

## TF2 Surveillance war room

In [24]:
#query recent sessions

query = """SELECT 
    s.session_id,
    p.name AS player_name,
    srv.address AS server_address,
    ss.name AS server_name,
    ss.current_map,
    s.score,
    s.duration,
    s.joined_at,
    s.left_at
FROM sessions AS s
JOIN players AS p ON s.player_id = p.player_id
JOIN servers AS srv ON s.server_id = srv.server_id
LEFT JOIN (
    SELECT 
        ss_inner.server_id, 
        ss_inner.name, 
        ss_inner.current_map,
        ss_inner.created_at AS latest_update
    FROM server_settings AS ss_inner
    WHERE ss_inner.created_at <= (SELECT MAX(ss_inner2.created_at)
                                  FROM server_settings AS ss_inner2
                                  WHERE ss_inner2.server_id = ss_inner.server_id)
    GROUP BY ss_inner.server_id
) AS ss ON srv.server_id = ss.server_id
ORDER BY s.session_id;
"""

all_sessions = pd.read_sql_query(query, conn)
all_sessions["duration"] = all_sessions["duration"].apply(format_duration)

print(all_sessions.shape)

all_sessions.tail()

(2, 9)


,session_id,player_name,server_address,server_name,current_map,score,duration,joined_at,left_at
0,24,max,51.161.199.24:27025,jump.tf (Sydney) | All Maps #1 | Tempus Network,jump_diamant_rc5,0,00h 05m 09s,2023-11-08 16:39:25,2023-11-08 16:44:34
1,29,max,103.25.57.21:27015,Jump (Adelaide) | All Maps | Tempus Network,jump_m0nkey_a5,0,00h 00m 51s,2023-11-08 16:44:44,2023-11-08 16:45:35


In [27]:
#Sessions by player name
player_name = 'Owlenstein™'
query = f"""SELECT  s.session_id, p.name AS player_name, srv.address AS server_address, ss.name AS server_name, ss.current_map, s.score, s.duration, s.joined_at, s.left_at
FROM sessions AS s
JOIN players AS p ON s.player_id = p.player_id
JOIN servers AS srv ON s.server_id = srv.server_id
LEFT JOIN (
    SELECT ss_inner.server_id, ss_inner.name, ss_inner.current_map,ss_inner.created_at AS latest_update
    FROM server_settings AS ss_inner
    WHERE ss_inner.created_at <= (SELECT MAX(ss_inner2.created_at)
                                  FROM server_settings AS ss_inner2
                                  WHERE ss_inner2.server_id = ss_inner.server_id)
    GROUP BY ss_inner.server_id
) AS ss ON srv.server_id = ss.server_id
WHERE p.name = ?
ORDER BY s.session_id;
"""
sessions_by_player = pd.read_sql_query(query, conn, params=[player_name])

sessions_by_player["duration"] = sessions_by_player["duration"].apply(format_duration)

sessions_by_player.head()

,session_id,player_name,server_address,server_name,current_map,score,duration,joined_at,left_at


In [43]:
#query players total playtime and sessions

query = """SELECT p.player_id, p.name, COUNT(s.session_id) AS session_count, SUM(s.duration) AS total_duration FROM players p JOIN sessions s ON p.player_id = s.player_id GROUP BY p.player_id, p.name HAVING COUNT(s.session_id) > 1;"""
session_count_by_player = pd.read_sql_query(query, conn)
session_count_by_player["total_duration"] = session_count_by_player["total_duration"].apply(format_duration)

print(session_count_by_player.shape)

session_count_by_player.head()

(779, 4)


,player_id,name,session_count,total_duration
0,1,CrySomeMore,8,138h 53m 46s
1,2,Humans Are Weak,9,137h 16m 21s
2,3,C++,6,108h 14m 40s
3,4,Nobody,7,131h 58m 18s
4,5,RageQuit,9,146h 23m 07s


In [46]:
#query player events by name

query = """
SELECT pe.*, s.name AS server_name, s.address AS server_address
FROM player_events AS pe
JOIN players AS p ON pe.player_id = p.player_id
JOIN servers AS s ON pe.server_id = s.server_id
WHERE p.name = 'elmaxo';
"""
events_by_name = pd.read_sql_query(query, conn)

print(events_by_name.shape)

events_by_name.head()

(2, 7)


,event_id,server_id,player_id,event_type,created_at,server_name,server_address
0,22455,3,399538,target join,2023-11-06 14:32:12,Uncletopia | Sydney | 2 | All Maps,51.161.198.97:27025
1,22663,3,399538,target join,2023-11-06 14:32:54,Uncletopia | Sydney | 2 | All Maps,51.161.198.97:27025
